# DX 704 Week 1 Project

This week's project will build a portfolio risk and return model, and make investing recommendations for hypothetical clients.
You will collect historical data, estimate returns and risks, construct efficient frontier portfolios, and sanity check the certainty of the maximum return portfolio.

The full project description and a template notebook are available on GitHub at the following link.

https://github.com/bu-cds-dx704/dx704-project-01


Feel free to use optimization tools or libraries (such as CVXOPT or scipy.optimize) to perform any calculations required for this mini project.

### Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Data

Collect historical monthly price data for the last 24 months covering 6 different stocks.
The data should cover 24 consecutive months including the last month that ended before this week's material was released on Blackboard.
To be clear, if a month ends between the Blackboard release and submitting your project, you do not need to add that month.

The six different stocks must include AAPL, SPY and TSLA.
At least one of the remaining 3 tickers must start with the same letter as your last name (e.g. professor Considine could use COIN).
This is to encourage diversity in what stocks you analyze; if you discuss this project with classmates, please make sure that you pick different tickers to differentiate your work.
Do not pick stocks with fewer than 24 consecutive months of price data.

In [32]:
# Imports
import yfinance as yf
import pandas as pd
from datetime import datetime
import calendar
import numpy as np
import os
from scipy.optimize import minimize

In [33]:
# Tickers - must include AAPL, SPY, TSLA
# The last 3 chosen start with M (for Mufti :D )
ALL_TICKERS = [
    'AAPL',  # Apple (required)
    'SPY',   # S&P 500 (required)
    'TSLA',  # Tesla (required)
    'MSFT',  # Microsoft
    'M',     # Macy's
    'MCD'    # McDonald's
]

# Date range: 24 consecutive months ending August 2025
START_MONTH = 9   # September
START_YEAR = 2023
END_MONTH = 8     # August
END_YEAR = 2025

print(f"Configuration:")
print(f"Tickers: {ALL_TICKERS}")
print(f"Date range: {START_MONTH}/{START_YEAR} to {END_MONTH}/{END_YEAR}")
print(f"Expected months: {(END_YEAR - START_YEAR) * 12 + (END_MONTH - START_MONTH) + 1}")

# Data collection functions
# Generates a list of end-month dates for the defined set range
def get_end_month_dates(start_year, start_month, end_year, end_month):
    dates = []
    current_year, current_month = start_year, start_month
    
    while (current_year, current_month) <= (end_year, end_month):
        # Get last day of month
        last_day = calendar.monthrange(current_year, current_month)[1]
        date = datetime(current_year, current_month, last_day)
        dates.append(date)
        
        # Move to next month
        current_month += 1
        if current_month > 12:
            current_month = 1
            current_year += 1
    
    return dates

# Gets the stock price data for the defined target dates
def get_stock_data_for_dates(ticker, target_dates):
    print(f"Getting data for {ticker}")
    
    # Download data with buffer around our date range
    start_date = target_dates[0].replace(day=1)  # Start of first month
    end_date = target_dates[-1]
    
    try:
        # Download stock data - explicitly set auto_adjust to avoid warning
        stock_data = yf.download(ticker, start=start_date, end=end_date, 
                               progress=False, auto_adjust=False)
        
        if stock_data.empty:
            print(f"PROBLEM: No data returned for {ticker}")
            return [None] * len(target_dates), [None] * len(target_dates)
        
        # Debug: print column names to see what we got
        print(f"  Columns for {ticker}: {list(stock_data.columns)}")
        
        prices = []
        actual_dates = []
        
        # Find the last available trading day of the month
        for target_date in target_dates:
            found_price = None
            found_date = None
            
            # Look back up to 5 days for trading day
            for days_back in range(5):  
                check_date = target_date - pd.Timedelta(days=days_back)
                
                if check_date.date() in [d.date() for d in stock_data.index]:
                    # Once a trading day is found
                    trading_day = [d for d in stock_data.index if d.date() == check_date.date()][0]
                    
                    # Try different possible column names for adjusted close
                    adj_close_value = None
                    if 'Adj Close' in stock_data.columns:
                        adj_close_value = stock_data.loc[trading_day, 'Adj Close']
                    elif ('Adj Close', ticker) in stock_data.columns:
                        adj_close_value = stock_data.loc[trading_day, ('Adj Close', ticker)]
                    elif len(stock_data.columns) == 6:  # Standard OHLCV + Adj Close
                        adj_close_value = stock_data.iloc[stock_data.index.get_loc(trading_day), 5]  # Usually last column
                    else:
                        # Fall back to regular Close if Adj Close not available
                        if 'Close' in stock_data.columns:
                            adj_close_value = stock_data.loc[trading_day, 'Close']
                        elif ('Close', ticker) in stock_data.columns:
                            adj_close_value = stock_data.loc[trading_day, ('Close', ticker)]
                    
                    if adj_close_value is not None:
                        found_price = round(float(adj_close_value), 2)
                        found_date = trading_day.strftime('%Y-%m-%d')
                        break
            
            if found_price is None:
                print(f"PROBLEM: No trading data found for {ticker} around {target_date.strftime('%Y-%m-%d')}")
            
            prices.append(found_price)
            actual_dates.append(found_date)

        return prices, actual_dates
        
    except Exception as e:
        print(f"ERROR getting data for {ticker}: {e}")
        print(f"  Exception type: {type(e)}")
        return [None] * len(target_dates), [None] * len(target_dates)

print("Functions completed")

Configuration:
Tickers: ['AAPL', 'SPY', 'TSLA', 'MSFT', 'M', 'MCD']
Date range: 9/2023 to 8/2025
Expected months: 24
Functions completed


Save the data as a TSV file named "historical_prices.tsv" and include a header row with the column names "date" and the 6 stock ticker symbols.
The date should be the last trading day of the month, so it may not be the last day of the month.
For example, the last trading day of November 2024 was 2024-11-29.
The remaining columns should contain the adjusted closing prices of the corresponding stock tickers on that day.


In [34]:
# YOUR CHANGES HERE

# Using our functions we made lets get our stock data and save it to a dataframe with the specified format
print("Stock Data Collector")
print("=" * 50)

# Get target dates (month ends)
target_dates = get_end_month_dates(START_YEAR, START_MONTH, END_YEAR, END_MONTH)
print(f"\nTarget dates generated: {len(target_dates)} months")
print(f"First date: {target_dates[0].strftime('%Y-%m-%d')}")
print(f"Last date: {target_dates[-1].strftime('%Y-%m-%d')}")

print("\n" + "-" * 50)

# Initialize results dictionary
results = {}

# Collect data for each ticker
for ticker in ALL_TICKERS:
    prices, dates = get_stock_data_for_dates(ticker, target_dates)
    results[ticker] = prices
    
    # Use dates from first successful ticker for the date column
    if 'date' not in results and dates[0] is not None:
        results['date'] = dates

# Create DataFrame
df = pd.DataFrame(results)

# Reorder columns to have date first
columns = ['date'] + ALL_TICKERS
df = df[columns]

print("\n" + "=" * 50)

# Display results preview
print("\nCollected Data Preview:")
print("First 5 rows:")
print(df.head())
print("\nLast 5 rows:")
print(df.tail())

# Save to TSV file
output_file = "historical_prices.tsv"
df.to_csv(output_file, sep='\t', index=False)
print(f"\nData saved to: {output_file}")

# Data quality check
print(f"\nData Quality Summary:")
print(f"Total months collected: {len(df)}")
for ticker in ALL_TICKERS:
    valid_count = df[ticker].notna().sum()
    percentage = (valid_count/len(df)*100) if len(df) > 0 else 0
    print(f"  {ticker}: {valid_count}/{len(df)} valid prices ({percentage:.1f}%)")

# Verify file creation
if os.path.exists(output_file):
    file_size = os.path.getsize(output_file)
    print(f"\nFile verification: {output_file} created successfully ({file_size} bytes)")
else:
    print(f"\nERROR: {output_file} was not created")

Stock Data Collector

Target dates generated: 24 months
First date: 2023-09-30
Last date: 2025-08-31

--------------------------------------------------
Getting data for AAPL
  Columns for AAPL: [('Adj Close', 'AAPL'), ('Close', 'AAPL'), ('High', 'AAPL'), ('Low', 'AAPL'), ('Open', 'AAPL'), ('Volume', 'AAPL')]
Getting data for SPY
  Columns for SPY: [('Adj Close', 'SPY'), ('Close', 'SPY'), ('High', 'SPY'), ('Low', 'SPY'), ('Open', 'SPY'), ('Volume', 'SPY')]
Getting data for TSLA


/tmp/ipykernel_2148/2996265075.py:95: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  found_price = round(float(adj_close_value), 2)
/tmp/ipykernel_2148/2996265075.py:95: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  found_price = round(float(adj_close_value), 2)
/tmp/ipykernel_2148/2996265075.py:95: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  found_price = round(float(adj_close_value), 2)
/tmp/ipykernel_2148/2996265075.py:95: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  found_price = round(float(adj_close_value), 2)


  Columns for TSLA: [('Adj Close', 'TSLA'), ('Close', 'TSLA'), ('High', 'TSLA'), ('Low', 'TSLA'), ('Open', 'TSLA'), ('Volume', 'TSLA')]
Getting data for MSFT
  Columns for MSFT: [('Adj Close', 'MSFT'), ('Close', 'MSFT'), ('High', 'MSFT'), ('Low', 'MSFT'), ('Open', 'MSFT'), ('Volume', 'MSFT')]
Getting data for M
  Columns for M: [('Adj Close', 'M'), ('Close', 'M'), ('High', 'M'), ('Low', 'M'), ('Open', 'M'), ('Volume', 'M')]
Getting data for MCD
  Columns for MCD: [('Adj Close', 'MCD'), ('Close', 'MCD'), ('High', 'MCD'), ('Low', 'MCD'), ('Open', 'MCD'), ('Volume', 'MCD')]


Collected Data Preview:
First 5 rows:
         date    AAPL     SPY    TSLA    MSFT      M     MCD
0  2023-09-29  169.55  417.87  250.22  311.06  10.74  251.35
1  2023-10-31  169.11  408.79  200.84  333.09  11.27  250.13
2  2023-11-30  188.36  446.14  240.08  374.04  14.68  270.51
3  2023-12-29  190.91  466.50  248.48  371.21  18.78  284.59
4  2024-01-31  182.85  473.93  187.29  392.47  17.07  280.96

Last 5 rows:
  

/tmp/ipykernel_2148/2996265075.py:95: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  found_price = round(float(adj_close_value), 2)
/tmp/ipykernel_2148/2996265075.py:95: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  found_price = round(float(adj_close_value), 2)


Submit "historical_prices.tsv" in Gradescope.

## Part 2: Calculate Historical Asset Returns

Calculate the historical asset returns based on the price data that you previously collected.

In [35]:
# YOUR CHANGES HERE
# Read the price data we created in Part 1
price_data = pd.read_csv("historical_prices.tsv", sep='\t')

print("Price data loaded:")
print(f"Shape: {price_data.shape}")
print("First few rows:")
print(price_data.head())

# Get all ticker columns (exclude the date column)
tickers = [col for col in price_data.columns if col != 'date']
print(f"\nCalculating returns for: {tickers}")

# Use pandas pct_change() method to calculate returns
# This calculates (current - previous) / previous automatically
# From lecture we want (ending price - starting price)/ starting price
price_returns = price_data[tickers].pct_change()

# Drop the first row since it will be NaN (no previous price)
price_returns = price_returns.iloc[1:]

# Add the dates (also drop first date to match)
dates = price_data['date'].iloc[1:].reset_index(drop=True)

# Create returns DataFrame
returns_df = pd.DataFrame()
returns_df['date'] = dates

# Add the ticker returns
for ticker in tickers:
    returns_df[ticker] = price_returns[ticker].reset_index(drop=True).round(6)

print(f"\nReturns calculated:")
print(f"Shape: {returns_df.shape}")
print("First few rows:")
print(returns_df.head())
print("\nLast few rows:")
print(returns_df.tail())

Price data loaded:
Shape: (24, 7)
First few rows:
         date    AAPL     SPY    TSLA    MSFT      M     MCD
0  2023-09-29  169.55  417.87  250.22  311.06  10.74  251.35
1  2023-10-31  169.11  408.79  200.84  333.09  11.27  250.13
2  2023-11-30  188.36  446.14  240.08  374.04  14.68  270.51
3  2023-12-29  190.91  466.50  248.48  371.21  18.78  284.59
4  2024-01-31  182.85  473.93  187.29  392.47  17.07  280.96

Calculating returns for: ['AAPL', 'SPY', 'TSLA', 'MSFT', 'M', 'MCD']

Returns calculated:
Shape: (23, 7)
First few rows:
         date      AAPL       SPY      TSLA      MSFT         M       MCD
0  2023-10-31 -0.002595 -0.021729 -0.197346  0.070822  0.049348 -0.004854
1  2023-11-30  0.113831  0.091367  0.195379  0.122940  0.302573  0.081478
2  2023-12-29  0.013538  0.045636  0.034988 -0.007566  0.279292  0.052050
3  2024-01-31 -0.042219  0.015927 -0.246257  0.057272 -0.091054 -0.012755
4  2024-02-29 -0.018540  0.052202  0.077901  0.042322 -0.046280  0.004164

Last few rows:
  

Save the data as a TSV file named "historical_returns.tsv" and include a header row with the column names "date" and the 6 stock ticker symbols.
Each row should have the date at the end of the month and the corresponding *relative* price changes.
For example, if the previous price was \$100 and the new price is \$110, the return value should be 0.10.
There should only be 23 rows of data in this file, since they are computed as the differences of 24 prices.

In [36]:
# YOUR CHANGES HERE

# Save to TSV file
output_file = "historical_returns.tsv"
returns_df.to_csv(output_file, sep='\t', index=False)

print(f"Returns data saved to: {output_file}")

# Verify the file and show summary
print(f"\nSummary:")
print(f"Total return periods: {len(returns_df)} (should be 23)")
print(f"Date range: {returns_df.iloc[0]['date']} to {returns_df.iloc[-1]['date']}")

# Show sample calculations for verification
print(f"\nSample return calculations:")
for i in range(min(3, len(returns_df))):
    print(f"\nMonth {i+1} ({returns_df.iloc[i]['date']}):")
    for ticker in tickers:
        if i < len(price_data) - 1:
            prev_price = price_data.iloc[i][ticker]
            curr_price = price_data.iloc[i+1][ticker] 
            calculated_return = returns_df.iloc[i][ticker]
            print(f"  {ticker}: ${prev_price:.2f} -> ${curr_price:.2f} = {calculated_return:.4f}")

# Data quality check (checking calculation work here)
print(f"\nData Quality Summary:")
for ticker in tickers:
    valid_count = returns_df[ticker].notna().sum()
    percentage = (valid_count/len(returns_df)*100) if len(returns_df) > 0 else 0
    print(f"  {ticker}: {valid_count}/{len(returns_df)} valid returns ({percentage:.1f}%)")

# File verification
if os.path.exists(output_file):
    file_size = os.path.getsize(output_file)
    print(f"\nFile verification: {output_file} created successfully ({file_size} bytes)")
else:
    print(f"\nERROR: {output_file} was not created")

Returns data saved to: historical_returns.tsv

Summary:
Total return periods: 23 (should be 23)
Date range: 2023-10-31 to 2025-08-29

Sample return calculations:

Month 1 (2023-10-31):
  AAPL: $169.55 -> $169.11 = -0.0026
  SPY: $417.87 -> $408.79 = -0.0217
  TSLA: $250.22 -> $200.84 = -0.1973
  MSFT: $311.06 -> $333.09 = 0.0708
  M: $10.74 -> $11.27 = 0.0493
  MCD: $251.35 -> $250.13 = -0.0049

Month 2 (2023-11-30):
  AAPL: $169.11 -> $188.36 = 0.1138
  SPY: $408.79 -> $446.14 = 0.0914
  TSLA: $200.84 -> $240.08 = 0.1954
  MSFT: $333.09 -> $374.04 = 0.1229
  M: $11.27 -> $14.68 = 0.3026
  MCD: $250.13 -> $270.51 = 0.0815

Month 3 (2023-12-29):
  AAPL: $188.36 -> $190.91 = 0.0135
  SPY: $446.14 -> $466.50 = 0.0456
  TSLA: $240.08 -> $248.48 = 0.0350
  MSFT: $374.04 -> $371.21 = -0.0076
  M: $14.68 -> $18.78 = 0.2793
  MCD: $270.51 -> $284.59 = 0.0520

Data Quality Summary:
  AAPL: 23/23 valid returns (100.0%)
  SPY: 23/23 valid returns (100.0%)
  TSLA: 23/23 valid returns (100.0%)
  MS

Submit "historical_returns.tsv" in Gradescope.

## Part 3: Estimate Returns

Estimate the expected returns for each asset using the previously calculated return data.
Just compute the average (mean) return for each asset over your data set; do not use other estimators that have been mentioned.
This will serve as your estimate of expected return for each asset.

In [37]:
# YOUR CHANGES HERE

# Read the returns data we created in Part 2
returns_data = pd.read_csv("historical_returns.tsv", sep='\t')

print("Returns data loaded:")
print(f"Shape: {returns_data.shape}")
print("First few rows:")
print(returns_data.head())

# Get all ticker columns (exclude the date column)
tickers = [col for col in returns_data.columns if col != 'date']
print(f"\nCalculating expected returns for: {tickers}")

# Calculate the mean return for each asset
estimated_returns = {}

for ticker in tickers:
    mean_return = returns_data[ticker].mean()
    estimated_returns[ticker] = mean_return
    print(f"{ticker}: {mean_return:.6f} ({mean_return*100:.4f}% per month)")

print(f"\nEstimated returns calculated for {len(estimated_returns)} assets")

Returns data loaded:
Shape: (23, 7)
First few rows:
         date      AAPL       SPY      TSLA      MSFT         M       MCD
0  2023-10-31 -0.002595 -0.021729 -0.197346  0.070822  0.049348 -0.004854
1  2023-11-30  0.113831  0.091367  0.195379  0.122940  0.302573  0.081478
2  2023-12-29  0.013538  0.045636  0.034988 -0.007566  0.279292  0.052050
3  2024-01-31 -0.042219  0.015927 -0.246257  0.057272 -0.091054 -0.012755
4  2024-02-29 -0.018540  0.052202  0.077901  0.042322 -0.046280  0.004164

Calculating expected returns for: ['AAPL', 'SPY', 'TSLA', 'MSFT', 'M', 'MCD']
AAPL: 0.015442 (1.5442% per month)
SPY: 0.019666 (1.9666% per month)
TSLA: 0.025313 (2.5313% per month)
MSFT: 0.023318 (2.3318% per month)
M: 0.014757 (1.4757% per month)
MCD: 0.010268 (1.0268% per month)

Estimated returns calculated for 6 assets


Save the estimated returns in a TSV file named "estimated_returns.tsv" and include a header row with the column names "asset" and "estimated_return".

In [38]:
# YOUR CHANGES HERE

# Create DataFrame with the required format
estimated_returns_df = pd.DataFrame([
    {'asset': asset, 'estimated_return': return_value}
    for asset, return_value in estimated_returns.items()
])

# Round to 6 decimal places for consistency
estimated_returns_df['estimated_return'] = estimated_returns_df['estimated_return'].round(6)

print("Estimated returns DataFrame:")
print(estimated_returns_df)

# Save to TSV file
output_file = "estimated_returns.tsv"
estimated_returns_df.to_csv(output_file, sep='\t', index=False)
print(f"\nEstimated returns saved to: {output_file}")

# Summary statistics
print(f"\nSummary:")
print(f"Number of assets: {len(estimated_returns_df)}")
print(f"Highest expected return: {estimated_returns_df['estimated_return'].max():.6f}")
print(f"Lowest expected return: {estimated_returns_df['estimated_return'].min():.6f}")
print(f"Average expected return: {estimated_returns_df['estimated_return'].mean():.6f}")

# File verification
if os.path.exists(output_file):
    file_size = os.path.getsize(output_file)
    print(f"\nFile verification: {output_file} created successfully ({file_size} bytes)")
else:
    print(f"\nERROR: {output_file} was not created")

Estimated returns DataFrame:
  asset  estimated_return
0  AAPL          0.015442
1   SPY          0.019666
2  TSLA          0.025313
3  MSFT          0.023318
4     M          0.014757
5   MCD          0.010268

Estimated returns saved to: estimated_returns.tsv

Summary:
Number of assets: 6
Highest expected return: 0.025313
Lowest expected return: 0.010268
Average expected return: 0.018127

File verification: estimated_returns.tsv created successfully (102 bytes)


Submit "estimated_returns.tsv" in Gradescope.

## Part 4: Estimate Risk

Estimate the covariance matrix for the asset returns to understand how the assets move together.

In [39]:
# YOUR CHANGES HERE

# Read the returns data we created in Part 2
returns_data = pd.read_csv("historical_returns.tsv", sep='\t')

print("Returns data loaded:")
print(f"Shape: {returns_data.shape}")

# Get all ticker columns (exclude the date column)
tickers = [col for col in returns_data.columns if col != 'date']
print(f"Calculating covariance matrix for: {tickers}")

# Extract just the return columns for covariance calculation
returns_only = returns_data[tickers]

# Calculate the covariance matrix
covariance_matrix = returns_only.cov()

print(f"\nCovariance matrix calculated:")
print(f"Shape: {covariance_matrix.shape}")
print("\nCovariance Matrix:")
print(covariance_matrix)

# Also calculate correlation matrix for reference (like in lecture notes)
correlation_matrix = returns_only.corr()
print(f"\nCorrelation Matrix (for reference):")
print(correlation_matrix)

# Calculate standard deviations for each asset (risk measure)
std_devs = returns_only.std()
print(f"\nStandard Deviations (Risk):")
for ticker in tickers:
    print(f"{ticker}: {std_devs[ticker]:.6f}")

Returns data loaded:
Shape: (23, 7)
Calculating covariance matrix for: ['AAPL', 'SPY', 'TSLA', 'MSFT', 'M', 'MCD']

Covariance matrix calculated:
Shape: (6, 6)

Covariance Matrix:
          AAPL       SPY      TSLA      MSFT         M       MCD
AAPL  0.003627  0.000833  0.003255  0.000389  0.002522  0.000600
SPY   0.000833  0.001292  0.002421  0.001488  0.002282  0.000123
TSLA  0.003255  0.002421  0.026703  0.002217  0.004953  0.001064
MSFT  0.000389  0.001488  0.002217  0.004044  0.002832 -0.000369
M     0.002522  0.002282  0.004953  0.002832  0.012837  0.000655
MCD   0.000600  0.000123  0.001064 -0.000369  0.000655  0.001819

Correlation Matrix (for reference):
          AAPL       SPY      TSLA      MSFT         M       MCD
AAPL  1.000000  0.384539  0.330787  0.101593  0.369599  0.233454
SPY   0.384539  1.000000  0.412117  0.650721  0.560266  0.080273
TSLA  0.330787  0.412117  1.000000  0.213353  0.267535  0.152634
MSFT  0.101593  0.650721  0.213353  1.000000  0.393113 -0.136049
M  

Save the estimated covariances to a TSV file named "estimated_covariance.tsv".
The header row should have a blank column name followed by the names of the assets.
Each data row should start with the name of an asset for that row, and be followed by the individual covariances corresponding to that row and column's assets.
(This is the format of pandas's `to_csv` method with `sep="\t"` when used on a covariance matrix as computed in the examples.)

In [40]:
# YOUR CHANGES HERE

# Saving the covariance matrix to TSV file
# Using pandas to_csv with sep='\t' as it automatically includes row names as first column
output_file = "estimated_covariance.tsv"
covariance_matrix.to_csv(output_file, sep='\t')

print(f"Covariance matrix saved to: {output_file}")

# Display the saved format
print(f"\nSaved covariance matrix format:")
saved_data = pd.read_csv(output_file, sep='\t', index_col=0)
print(saved_data)

# File verification and summary
if os.path.exists(output_file):
    file_size = os.path.getsize(output_file)
    print(f"\nFile verification: {output_file} created successfully ({file_size} bytes)")
else:
    print(f"\nERROR: {output_file} was not created")

# Summary of covariance matrix properties
print(f"\nCovariance Matrix Summary:")
print(f"Dimensions: {covariance_matrix.shape[0]}x{covariance_matrix.shape[1]}")
print(f"Diagonal (variances):")
for ticker in tickers:
    variance = covariance_matrix.loc[ticker, ticker]
    std_dev = np.sqrt(variance)
    print(f"  {ticker}: {variance:.6f} (std dev: {std_dev:.6f})")

# Find highest and lowest correlations (off-diagonal elements)
corr_values = []
for i in range(len(tickers)):
    for j in range(i+1, len(tickers)):
        corr_values.append((tickers[i], tickers[j], correlation_matrix.iloc[i,j]))

corr_values.sort(key=lambda x: x[2])
print(f"\nHighest correlation: {corr_values[-1][0]} & {corr_values[-1][1]}: {corr_values[-1][2]:.4f}")
print(f"Lowest correlation: {corr_values[0][0]} & {corr_values[0][1]}: {corr_values[0][2]:.4f}")

Covariance matrix saved to: estimated_covariance.tsv

Saved covariance matrix format:
          AAPL       SPY      TSLA      MSFT         M       MCD
AAPL  0.003627  0.000833  0.003255  0.000389  0.002522  0.000600
SPY   0.000833  0.001292  0.002421  0.001488  0.002282  0.000123
TSLA  0.003255  0.002421  0.026703  0.002217  0.004953  0.001064
MSFT  0.000389  0.001488  0.002217  0.004044  0.002832 -0.000369
M     0.002522  0.002282  0.004953  0.002832  0.012837  0.000655
MCD   0.000600  0.000123  0.001064 -0.000369  0.000655  0.001819

File verification: estimated_covariance.tsv created successfully (836 bytes)

Covariance Matrix Summary:
Dimensions: 6x6
Diagonal (variances):
  AAPL: 0.003627 (std dev: 0.060223)
  SPY: 0.001292 (std dev: 0.035950)
  TSLA: 0.026703 (std dev: 0.163412)
  MSFT: 0.004044 (std dev: 0.063592)
  M: 0.012837 (std dev: 0.113300)
  MCD: 0.001819 (std dev: 0.042644)

Highest correlation: SPY & MSFT: 0.6507
Lowest correlation: MSFT & MCD: -0.1360


Submit "estimated_covariance.tsv" in Gradescope.

## Part 5: Construct the Maximum Return Portfolio

Compute the maximum return portfolio based on your previously estimated risks and returns.

In [41]:
# YOUR CHANGES HERE

# Read the estimated returns we calculated in Part 3
estimated_returns = pd.read_csv("estimated_returns.tsv", sep='\t')

print("Estimated returns loaded:")
print(estimated_returns)

# Find the asset with the highest expected return
max_return_idx = estimated_returns['estimated_return'].idxmax()
max_return_asset = estimated_returns.loc[max_return_idx, 'asset']
max_return_value = estimated_returns.loc[max_return_idx, 'estimated_return']

print(f"\nAsset with highest expected return:")
print(f"{max_return_asset}: {max_return_value:.6f}")

# According to lecture notes, under standard investment constraints
# (budget constraint: weights sum to 1, non-negativity: all weights >= 0)
# the maximum return portfolio puts 100% in the highest return asset

# Create maximum return portfolio
max_return_portfolio = []
for _, row in estimated_returns.iterrows():
    asset = row['asset']
    if asset == max_return_asset:
        allocation = 1.0  # 100% allocation to highest return asset
    else:
        allocation = 0.0  # 0% allocation to all other assets
    
    max_return_portfolio.append({
        'asset': asset,
        'allocation': allocation
    })

# Convert to DataFrame
max_return_df = pd.DataFrame(max_return_portfolio)

print(f"\nMaximum Return Portfolio:")
print(max_return_df)

# Verify constraints
total_allocation = max_return_df['allocation'].sum()
all_non_negative = (max_return_df['allocation'] >= 0).all()

print(f"\nConstraint verification:")
print(f"Total allocation: {total_allocation:.6f} (should be 1.0)")
print(f"All allocations non-negative: {all_non_negative}")

# Calculate expected portfolio return
# Create a dictionary for easier lookup
returns_dict = dict(zip(estimated_returns['asset'], estimated_returns['estimated_return']))

# Calculate portfolio return: sum of (weight * return) for each asset
portfolio_return = sum(max_return_df['allocation'] * max_return_df['asset'].map(returns_dict))
print(f"Expected portfolio return: {portfolio_return:.6f}")

Estimated returns loaded:
  asset  estimated_return
0  AAPL          0.015442
1   SPY          0.019666
2  TSLA          0.025313
3  MSFT          0.023318
4     M          0.014757
5   MCD          0.010268

Asset with highest expected return:
TSLA: 0.025313

Maximum Return Portfolio:
  asset  allocation
0  AAPL         0.0
1   SPY         0.0
2  TSLA         1.0
3  MSFT         0.0
4     M         0.0
5   MCD         0.0

Constraint verification:
Total allocation: 1.000000 (should be 1.0)
All allocations non-negative: True
Expected portfolio return: 0.025313


Save the maximum return portfolio in a TSV file named "maximum_return.tsv".
The header row should have two columns, "asset" and "allocation".
The allocation values should sum up to one.


In [42]:
# YOUR CHANGES HERE

# Save the maximum return portfolio to TSV file
output_file = "maximum_return.tsv"
max_return_df.to_csv(output_file, sep='\t', index=False)

print(f"Maximum return portfolio saved to: {output_file}")

# Display the saved format
print(f"\nSaved portfolio format:")
saved_portfolio = pd.read_csv(output_file, sep='\t')
print(saved_portfolio)

# File verification
if os.path.exists(output_file):
    file_size = os.path.getsize(output_file)
    print(f"\nFile verification: {output_file} created successfully ({file_size} bytes)")
else:
    print(f"\nERROR: {output_file} was not created")

# Portfolio summary
print(f"\nPortfolio Summary:")
print(f"Strategy: Maximum return (100% in highest return asset)")
print(f"Selected asset: {max_return_asset}")
print(f"Expected monthly return: {portfolio_return:.6f} ({portfolio_return*100:.4f}%)")
print(f"Number of assets in portfolio: {(max_return_df['allocation'] > 0).sum()}")

Maximum return portfolio saved to: maximum_return.tsv

Saved portfolio format:
  asset  allocation
0  AAPL         0.0
1   SPY         0.0
2  TSLA         1.0
3  MSFT         0.0
4     M         0.0
5   MCD         0.0

File verification: maximum_return.tsv created successfully (66 bytes)

Portfolio Summary:
Strategy: Maximum return (100% in highest return asset)
Selected asset: TSLA
Expected monthly return: 0.025313 (2.5313%)
Number of assets in portfolio: 1


Submit "maximum_return.tsv" in Gradescope.

## Part 6: Construct the Minimum Risk Portfolio

Compute the minimum risk portfolio based on your previously estimated risks.

In [43]:
# YOUR CHANGES HERE

# Load the covariance matrix from Part 4
covariance_matrix = pd.read_csv("estimated_covariance.tsv", sep='\t', index_col=0)

print("Covariance matrix loaded:")
print(covariance_matrix)

# Get asset names
assets = list(covariance_matrix.columns)
n_assets = len(assets)

print(f"\nAssets: {assets}")
print(f"Number of assets: {n_assets}")

# Convert covariance matrix to numpy array for optimization
cov_array = covariance_matrix.values

print(f"\nCovariance matrix shape: {cov_array.shape}")

# Define the objective function: minimize portfolio variance
# Portfolio variance = w^T * Σ * w (where w is weights, Σ is covariance matrix)
def portfolio_variance(weights, cov_matrix):
    return np.dot(weights.T, np.dot(cov_matrix, weights))

# Define constraints
# Budget constraint: sum of weights = 1
constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1.0}]

# Non-negativity constraints: all weights >= 0
bounds = [(0, 1) for _ in range(n_assets)]

# Initial guess: equal weights
initial_weights = np.array([1/n_assets] * n_assets)

print(f"\nInitial weights (equal allocation): {initial_weights}")
print(f"Initial portfolio variance: {portfolio_variance(initial_weights, cov_array):.6f}")

# Solve the optimization problem
result = minimize(
    portfolio_variance,
    initial_weights,
    args=(cov_array,),
    method='SLSQP',
    bounds=bounds,
    constraints=constraints
)

# Extract optimal weights
optimal_weights = result.x
min_variance = result.fun

print(f"\nOptimization result:")
print(f"Success: {result.success}")
print(f"Minimum variance: {min_variance:.6f}")
print(f"Minimum standard deviation (risk): {np.sqrt(min_variance):.6f}")

# Create minimum risk portfolio DataFrame
min_risk_portfolio = pd.DataFrame({
    'asset': assets,
    'allocation': optimal_weights
})

print(f"\nMinimum Risk Portfolio:")
print(min_risk_portfolio)

# Verify constraints
total_allocation = min_risk_portfolio['allocation'].sum()
all_non_negative = (min_risk_portfolio['allocation'] >= 0).all()

print(f"\nConstraint verification:")
print(f"Total allocation: {total_allocation:.6f} (should be 1.0)")
print(f"All allocations non-negative: {all_non_negative}")

Covariance matrix loaded:
          AAPL       SPY      TSLA      MSFT         M       MCD
AAPL  0.003627  0.000833  0.003255  0.000389  0.002522  0.000600
SPY   0.000833  0.001292  0.002421  0.001488  0.002282  0.000123
TSLA  0.003255  0.002421  0.026703  0.002217  0.004953  0.001064
MSFT  0.000389  0.001488  0.002217  0.004044  0.002832 -0.000369
M     0.002522  0.002282  0.004953  0.002832  0.012837  0.000655
MCD   0.000600  0.000123  0.001064 -0.000369  0.000655  0.001819

Assets: ['AAPL', 'SPY', 'TSLA', 'MSFT', 'M', 'MCD']
Number of assets: 6

Covariance matrix shape: (6, 6)

Initial weights (equal allocation): [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
Initial portfolio variance: 0.002801

Optimization result:
Success: True
Minimum variance: 0.000842
Minimum standard deviation (risk): 0.029022

Minimum Risk Portfolio:
  asset    allocation
0  AAPL  9.445829e-02
1   SPY  3.133991e-01
2  TSLA  2.255141e-17
3  MSFT  1.143472e-01
4     M  0.000000e+00
5   MCD

Save the minimum risk portfolio in a TSV file named "minimum_risk.tsv".
The header row should have two columns, "asset" and "allocation".
The allocation values should sum up to one.


In [44]:
# YOUR CHANGES HERE

# Round allocations
min_risk_portfolio['allocation'] = min_risk_portfolio['allocation'].round(6)

# Save the minimum risk portfolio to TSV file
output_file = "minimum_risk.tsv"
min_risk_portfolio.to_csv(output_file, sep='\t', index=False)

print(f"Minimum risk portfolio saved to: {output_file}")

# Display the saved format
print(f"\nSaved portfolio format:")
saved_portfolio = pd.read_csv(output_file, sep='\t')
print(saved_portfolio)

# File verification
if os.path.exists(output_file):
    file_size = os.path.getsize(output_file)
    print(f"\nFile verification: {output_file} created successfully ({file_size} bytes)")
else:
    print(f"\nERROR: {output_file} was not created")

# Calculate expected return of minimum risk portfolio
estimated_returns = pd.read_csv("estimated_returns.tsv", sep='\t')
returns_dict = dict(zip(estimated_returns['asset'], estimated_returns['estimated_return']))

portfolio_return = sum(min_risk_portfolio['allocation'] * min_risk_portfolio['asset'].map(returns_dict))

print(f"\nPortfolio Summary:")
print(f"Strategy: Minimum risk (variance minimization)")
print(f"Portfolio variance: {min_variance:.6f}")
print(f"Portfolio standard deviation: {np.sqrt(min_variance):.6f}")
print(f"Expected portfolio return: {portfolio_return:.6f} ({portfolio_return*100:.4f}%)")
print(f"Number of assets with positive allocation: {(min_risk_portfolio['allocation'] > 0.001).sum()}")

# Show which assets have meaningful allocations
print(f"\nAssets with allocation > 0.1%:")
for _, row in min_risk_portfolio.iterrows():
    if row['allocation'] > 0.001:
        print(f"  {row['asset']}: {row['allocation']:.4f} ({row['allocation']*100:.2f}%)")

Minimum risk portfolio saved to: minimum_risk.tsv

Saved portfolio format:
  asset  allocation
0  AAPL    0.094458
1   SPY    0.313399
2  TSLA    0.000000
3  MSFT    0.114347
4     M    0.000000
5   MCD    0.477795

File verification: minimum_risk.tsv created successfully (86 bytes)

Portfolio Summary:
Strategy: Minimum risk (variance minimization)
Portfolio variance: 0.000842
Portfolio standard deviation: 0.029022
Expected portfolio return: 0.015194 (1.5194%)
Number of assets with positive allocation: 4

Assets with allocation > 0.1%:
  AAPL: 0.0945 (9.45%)
  SPY: 0.3134 (31.34%)
  MSFT: 0.1143 (11.43%)
  MCD: 0.4778 (47.78%)


Submit "minimum_risk.tsv" in Gradescope.

## Part 7: Build Efficient Frontier Portfolios

Compute 101 portfolios along the mean-variance efficient frontier with evenly spaced estimated returns.
The first portfolio should be the minimum risk portfolio from part 4, and the last portfolio should be the maximum return portfolio from part 3.
The estimated return of each portfolio should be higher than the previous by one percent of the difference between the first and last portfolios.
That is, the estimated return of the portfolios should be similar to `np.linspace(min_risk_return, max_return, 101)`.


In [45]:
# YOUR CHANGES HERE

# Load required data
covariance_matrix = pd.read_csv("estimated_covariance.tsv", sep='\t', index_col=0)
estimated_returns = pd.read_csv("estimated_returns.tsv", sep='\t')
min_risk_portfolio = pd.read_csv("minimum_risk.tsv", sep='\t')
max_return_portfolio = pd.read_csv("maximum_return.tsv", sep='\t')

print("All dfs loaded")

# Get asset information
assets = list(covariance_matrix.columns)
n_assets = len(assets)
cov_array = covariance_matrix.values

# Create returns dictionary and array
returns_dict = dict(zip(estimated_returns['asset'], estimated_returns['estimated_return']))
returns_array = np.array([returns_dict[asset] for asset in assets])

print(f"Assets: {assets}")
print(f"Number of assets: {n_assets}")

# Calculate min and max portfolio returns
min_risk_return = sum(min_risk_portfolio['allocation'] * min_risk_portfolio['asset'].map(returns_dict))
max_return_value = sum(max_return_portfolio['allocation'] * max_return_portfolio['asset'].map(returns_dict))

print(f"\nMinimum risk portfolio return: {min_risk_return:.6f}")
print(f"Maximum return portfolio return: {max_return_value:.6f}")

# Create 101 evenly spaced target returns
target_returns = np.linspace(min_risk_return, max_return_value, 101)
print(f"\nTarget returns range: {target_returns[0]:.6f} to {target_returns[-1]:.6f}")
print(f"Number of portfolios: {len(target_returns)}")

# Define portfolio optimization functions
def portfolio_variance(weights, cov_matrix):
    return np.dot(weights.T, np.dot(cov_matrix, weights))

def portfolio_return(weights, returns):
    return np.dot(weights, returns)

# Store results
efficient_portfolios = []

for i, target_return in enumerate(target_returns):
    # Define constraints
    constraints = [
        {'type': 'eq', 'fun': lambda w: np.sum(w) - 1.0},  # Budget constraint
        {'type': 'eq', 'fun': lambda w, ret=target_return: portfolio_return(w, returns_array) - ret}  # Return constraint
    ]
    
    # Bounds for non-negativity
    bounds = [(0, 1) for _ in range(n_assets)]
    
    # Initial guess - use min risk portfolio for first, adjust for others
    if i == 0:
        initial_weights = min_risk_portfolio['allocation'].values
    elif i == len(target_returns) - 1:
        initial_weights = max_return_portfolio['allocation'].values
    else:
        # Linear interpolation between min risk and max return portfolios
        alpha = i / (len(target_returns) - 1)
        initial_weights = ((1 - alpha) * min_risk_portfolio['allocation'].values + 
                          alpha * max_return_portfolio['allocation'].values)
    
    # Solve optimization
    result = minimize(
        portfolio_variance,
        initial_weights,
        args=(cov_array,),
        method='SLSQP',
        bounds=bounds,
        constraints=constraints,
        options={'ftol': 1e-9, 'maxiter': 1000}
    )
    
    if result.success:
        weights = result.x
        portfolio_var = result.fun
        portfolio_std = np.sqrt(portfolio_var)
        actual_return = portfolio_return(weights, returns_array)
        
        # Store portfolio information
        portfolio_data = {
            'index': i,
            'return': actual_return,
            'risk': portfolio_std
        }
        
        # Add asset allocations
        for j, asset in enumerate(assets):
            portfolio_data[asset] = weights[j]
        
        efficient_portfolios.append(portfolio_data)
        
        if i % 20 == 0:  # Progress update every 20 portfolios
            print(f"Portfolio {i}: Return={actual_return:.6f}, Risk={portfolio_std:.6f}")
    
    else:
        print(f"Optimization failed for portfolio {i} with target return {target_return:.6f}")

print(f"\nOptimized {len(efficient_portfolios)} portfolios")

# Create DataFrame
efficient_frontier_df = pd.DataFrame(efficient_portfolios)

print(f"\nEfficient frontier DataFrame shape: {efficient_frontier_df.shape}")
print("First few portfolios:")
print(efficient_frontier_df.head())
print("\nLast few portfolios:")
print(efficient_frontier_df.tail())

All dfs loaded
Assets: ['AAPL', 'SPY', 'TSLA', 'MSFT', 'M', 'MCD']
Number of assets: 6

Minimum risk portfolio return: 0.015194
Maximum return portfolio return: 0.025313

Target returns range: 0.015194 to 0.025313
Number of portfolios: 101
Portfolio 0: Return=0.015194, Risk=0.028677


Portfolio 20: Return=0.017218, Risk=0.029441
Portfolio 40: Return=0.019242, Risk=0.034164
Portfolio 60: Return=0.021266, Risk=0.043433
Portfolio 80: Return=0.023289, Risk=0.060537
Portfolio 100: Return=0.025313, Risk=0.163412

Optimized 101 portfolios

Efficient frontier DataFrame shape: (101, 9)
First few portfolios:
   index    return      risk      AAPL       SPY          TSLA      MSFT  \
0      0  0.015194  0.028677  0.039058  0.456790  0.000000e+00  0.033047   
1      1  0.015295  0.028610  0.038079  0.465558  1.734723e-18  0.034875   
2      2  0.015397  0.028553  0.037099  0.474326  0.000000e+00  0.036703   
3      3  0.015498  0.028508  0.036120  0.483094  0.000000e+00  0.038531   
4      4  0.015599  0.028474  0.035218  0.491947  2.328324e-17  0.040266   

              M       MCD  
0  1.680513e-18  0.471104  
1  4.336809e-18  0.461488  
2  3.035766e-18  0.451872  
3  0.000000e+00  0.442255  
4  0.000000e+00  0.432568  

Last few portfolios:
     index    return      risk   

Save the portfolios in a TSV file named "efficient_frontier.tsv".
The header row should have columns "index", "return", "risk", and all the asset tickers.
Each data row should have the portfolio index (0-100), the estimated return of the portfolio, the estimated standard deviation (not variance) of the portfolio, and all the asset allocations (which should sum to one).

In [46]:
# YOUR CHANGES HERE

# Round values for cleaner output
efficient_frontier_df['return'] = efficient_frontier_df['return'].round(6)
efficient_frontier_df['risk'] = efficient_frontier_df['risk'].round(6)

# Round asset allocations
for asset in assets:
    efficient_frontier_df[asset] = efficient_frontier_df[asset].round(6)

# Verify allocations sum to 1
allocation_sums = efficient_frontier_df[assets].sum(axis=1)
print(f"\nAllocation verification:")
print(f"Min sum: {allocation_sums.min():.6f}")
print(f"Max sum: {allocation_sums.max():.6f}")
print(f"All sums close to 1.0: {np.allclose(allocation_sums, 1.0, atol=1e-5)}")

# Save to TSV file
output_file = "efficient_frontier.tsv"
efficient_frontier_df.to_csv(output_file, sep='\t', index=False)

print(f"\nEfficient frontier saved to: {output_file}")

# File verification
if os.path.exists(output_file):
    file_size = os.path.getsize(output_file)
    print(f"File verification: {output_file} created successfully ({file_size} bytes)")
else:
    print(f"ERROR: {output_file} was not created")

# Summary statistics
print(f"\nEfficient Frontier Summary:")
print(f"Number of portfolios: {len(efficient_frontier_df)}")
print(f"Return range: {efficient_frontier_df['return'].min():.6f} to {efficient_frontier_df['return'].max():.6f}")
print(f"Risk range: {efficient_frontier_df['risk'].min():.6f} to {efficient_frontier_df['risk'].max():.6f}")

# Show first portfolio (minimum risk)
print(f"\nFirst portfolio (minimum risk):")
first_portfolio = efficient_frontier_df.iloc[0]
print(f"Return: {first_portfolio['return']:.6f}, Risk: {first_portfolio['risk']:.6f}")
for asset in assets:
    if first_portfolio[asset] > 0.001:
        print(f"  {asset}: {first_portfolio[asset]:.4f}")

# Show last portfolio (maximum return)
print(f"\nLast portfolio (maximum return):")
last_portfolio = efficient_frontier_df.iloc[-1]
print(f"Return: {last_portfolio['return']:.6f}, Risk: {last_portfolio['risk']:.6f}")
for asset in assets:
    if last_portfolio[asset] > 0.001:
        print(f"  {asset}: {last_portfolio[asset]:.4f}")


Allocation verification:
Min sum: 0.999999
Max sum: 1.000001
All sums close to 1.0: True

Efficient frontier saved to: efficient_frontier.tsv
File verification: efficient_frontier.tsv created successfully (6108 bytes)

Efficient Frontier Summary:
Number of portfolios: 101
Return range: 0.015194 to 0.025313
Risk range: 0.028439 to 0.163412

First portfolio (minimum risk):
Return: 0.015194, Risk: 0.028677
  AAPL: 0.0391
  SPY: 0.4568
  MSFT: 0.0330
  MCD: 0.4711

Last portfolio (maximum return):
Return: 0.025313, Risk: 0.163412
  TSLA: 1.0000


Submit "efficient_frontier.tsv" in Gradescope.

## Part 8: Check Maximum Return Portfolio Stability

Check the stability of the maximum return portfolio by resampling the estimated risk/return model.

Repeat 1000 times -
1. Use `np.random.multivariate_normal` to generate 23 return samples using your previously estimated risks and returns.
2. Estimate the return of each asset using that resampled return history.
3. Check which asset had the highest return in those resampled estimates.

This procedure is a reduced and simplified version of the Michaud resampled efficient frontier procedure that takes uncertainty in the risk model into account.

In [47]:
# YOUR CHANGES HERE

# Load required data
estimated_returns = pd.read_csv("estimated_returns.tsv", sep='\t')
covariance_matrix = pd.read_csv("estimated_covariance.tsv", sep='\t', index_col=0)

print("Data loaded for stability analysis")

# Get assets and setup parameters
assets = list(covariance_matrix.columns)
n_assets = len(assets)
n_samples = 23  # Number of return observations we had
n_simulations = 1000

print(f"Assets: {assets}")
print(f"Running {n_simulations} simulations with {n_samples} samples each")

# Create mean returns vector and covariance matrix as numpy arrays
mean_returns = np.array([estimated_returns[estimated_returns['asset'] == asset]['estimated_return'].iloc[0] 
                        for asset in assets])
cov_matrix = covariance_matrix.values

print(f"\nOriginal estimated returns:")
for i, asset in enumerate(assets):
    print(f"  {asset}: {mean_returns[i]:.6f}")

# Track which asset has highest return in each simulation
highest_return_counts = {asset: 0 for asset in assets}

# Set random seed for reproducibility
np.random.seed(42)

for sim in range(n_simulations):
    # Generate 23 random return samples using multivariate normal distribution
    # Each row is a time period, each column is an asset
    simulated_returns = np.random.multivariate_normal(mean_returns, cov_matrix, n_samples)
    
    # Calculate mean return for each asset from the simulated data
    resampled_means = np.mean(simulated_returns, axis=0)
    
    # Find which asset has the highest resampled mean return
    highest_return_idx = np.argmax(resampled_means)
    highest_return_asset = assets[highest_return_idx]
    
    # Increment counter for this asset
    highest_return_counts[highest_return_asset] += 1
    
    # Progress update every 200 simulations
    if (sim + 1) % 200 == 0:
        print(f"Completed {sim + 1} simulations")

print(f"\nSimulation results:")
print(f"Times each asset had the highest return:")
for asset in assets:
    count = highest_return_counts[asset]
    probability = count / n_simulations
    print(f"  {asset}: {count} times ({probability:.3f} probability)")

# Calculate probabilities
probabilities = []
for asset in assets:
    prob = highest_return_counts[asset] / n_simulations
    probabilities.append({'asset': asset, 'probability': prob})

# Create results DataFrame
stability_results = pd.DataFrame(probabilities)
stability_results = stability_results.sort_values('probability', ascending=False)

print(f"\nStability Results (sorted by probability):")
print(stability_results)

Data loaded for stability analysis
Assets: ['AAPL', 'SPY', 'TSLA', 'MSFT', 'M', 'MCD']
Running 1000 simulations with 23 samples each

Original estimated returns:
  AAPL: 0.015442
  SPY: 0.019666
  TSLA: 0.025313
  MSFT: 0.023318
  M: 0.014757
  MCD: 0.010268
Completed 200 simulations
Completed 400 simulations
Completed 600 simulations
Completed 800 simulations
Completed 1000 simulations

Simulation results:
Times each asset had the highest return:
  AAPL: 75 times (0.075 probability)
  SPY: 47 times (0.047 probability)
  TSLA: 418 times (0.418 probability)
  MSFT: 269 times (0.269 probability)
  M: 148 times (0.148 probability)
  MCD: 43 times (0.043 probability)

Stability Results (sorted by probability):
  asset  probability
2  TSLA        0.418
3  MSFT        0.269
4     M        0.148
0  AAPL        0.075
1   SPY        0.047
5   MCD        0.043


Save a file "max_return_probabilities.tsv" with the distribution of highest return assets.
The header row should have columns "asset" and "probability".
There should be a data row for each asset and its sample probability of having the highest return based on those 1000 resampled estimates.


In [48]:
# YOUR CHANGES HERE

# Round probabilities to reasonable precision
stability_results['probability'] = stability_results['probability'].round(4)

# Save to TSV file
output_file = "max_return_probabilities.tsv"
stability_results.to_csv(output_file, sep='\t', index=False)

print(f"\nMaximum return probabilities saved to: {output_file}")

# Display the saved format
print(f"\nSaved probabilities:")
saved_data = pd.read_csv(output_file, sep='\t')
print(saved_data)

# File verification
if os.path.exists(output_file):
    file_size = os.path.getsize(output_file)
    print(f"\nFile verification: {output_file} created successfully ({file_size} bytes)")
else:
    print(f"\nERROR: {output_file} was not created")


Maximum return probabilities saved to: max_return_probabilities.tsv

Saved probabilities:
  asset  probability
0  TSLA        0.418
1  MSFT        0.269
2     M        0.148
3  AAPL        0.075
4   SPY        0.047
5   MCD        0.043

File verification: max_return_probabilities.tsv created successfully (79 bytes)


Submit "max_return_probabilities.tsv" in Gradescope.

## Part 9: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgements.txt" in Gradescope.

## Part 10: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

Submit "project.ipynb" in Gradescope.